### Launch EVA DB
Run the command `python eva.py` in the server where you want to deploy EVA

### Establish Connection With Eva DB

In [147]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

In [148]:
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

In [149]:
cursor = connection.cursor()

### Upload the survellience video to analyse

In [ ]:
cursor.execute('UPLOAD INFILE "../data/mnist/mnist.mp4" PATH "video.mp4";')
response = cursor.fetch_all()

### Visualize Video

In [155]:
from IPython.display import Video
Video("../data/mnist/mnist.mp4", embed=True)

### Load video into EVA

In [ ]:
cursor.execute('LOAD DATA INFILE "video.mp4" INTO MyVideo;')
response = cursor.fetch_all()
print(response)

Response Object:
@status: 0
@batch: Batch Object:
@dataframe:        Video
0  video.mp4
@batch_size: 1
@identifier_column: None
@metrics: None


In [ ]:
cursor.execute("""CREATE UDF IF NOT EXISTS MnistCNN
                  INPUT  (data NDARRAY (3, 28, 28))
                  OUTPUT (label TEXT(2))
                  TYPE  Classification
                  IMPL  'tutorials/apps/mnist/eva_mnist_udf.py';
        """)
response = cursor.fetch_all()
print(response)

Response Object:
@status: 0
@batch: Batch Object:
@dataframe: Empty DataFrame
Columns: []
Index: []
@batch_size: 0
@identifier_column: None
@metrics: None


### Run Object detector on the video

In [163]:
cursor.execute("""SELECT id, MnistCNN(data).label FROM MyVideo""")
response = cursor.fetch_all()
print(response)

Response Object:
@status: 0
@batch: Batch Object:
@dataframe:       myvideo.id  mnistcnn.label
0              0               6
1              1               6
2              2               6
3              3               6
4              4               6
...          ...             ...
1195        1195               6
1196        1196               6
1197        1197               6
1198        1198               6
1199        1199               6

[1200 rows x 2 columns]
@batch_size: 1200
@identifier_column: None
@metrics: None


### Visualize output of Object detector on the video

In [204]:
import cv2
def annotate_video(labels, input_video_path, output_video_path):
    color=(0,255,0)
    thickness=3

    vcap = cv2.VideoCapture(input_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G') #codec
    video=cv2.VideoWriter(output_path, fourcc, fps, (width,height))

    frame_id = 0
    # Capture frame-by-frame
    ret, frame = vcap.read()  # ret = 1 if the video is captured; frame is the image

    while ret:
        cv2.putText(frame, str(labels[frame_id]), (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1) # object label
        video.write(frame)

        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [205]:
from ipywidgets import Video
input_path = '../data/mnist/mnist.mp4'
output_path = '/tmp/annotated_video.avi'
annotate_video(response.batch.frames['mnistcnn.label'], input_path, output_path)
Video.from_file(output_path)

Video(value=b'RIFF\x9a\\\x0b\x00AVI LIST\xec\x11\x00\x00hdrlavih8\x00\x00\x005\x82\x00\x00\xe8D\x00\x00\x00\x0…

In [203]:
from IPython.display import Video
Video(output_path)

In [199]:
from ipywidgets import Video
Video.from_file(output_path)

Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x00\x01\x89\x88mdat\x00…

AttributeError: 'bytes' object has no attribute 'encode'